<a href="https://colab.research.google.com/github/bengisuycl/CMP719_TERM_PROJECT/blob/main/added_histogram_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install torch torchvision matplotlib scikit-image tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# Dataset path
rgb_path = '/content/drive/MyDrive/datasets/rgb_images'
ir_path  = '/content/drive/MyDrive/datasets/IR'

# Target dataset directories
os.makedirs('/content/drive/MyDrive/datasets/rgb2ir/trainA', exist_ok=True)
os.makedirs('/content/drive/MyDrive/datasets/rgb2ir/trainB', exist_ok=True)
os.makedirs('/content/drive/MyDrive/datasets/rgb2ir/testA', exist_ok=True)
os.makedirs('/content/drive/MyDrive/datasets/rgb2ir/testB', exist_ok=True)


Mounted at /content/drive


In [ ]:
import glob, random, shutil

def split_and_copy_images(src_dir, train_dir, test_dir, split_ratio=0.8, limit=None):
    images = glob.glob(os.path.join(src_dir, '*.jpg')) + glob.glob(os.path.join(src_dir, '*.png'))
    if limit:
        images = random.sample(images, min(limit, len(images)))
    random.shuffle(images)
    split_point = int(len(images) * split_ratio)
    train_images = images[:split_point]
    test_images  = images[split_point:]

    for img in train_images:
        shutil.copy(img, train_dir)
    for img in test_images:
        shutil.copy(img, test_dir)

# RGB images: limit 5000 for balanced training
split_and_copy_images(rgb_path, '/content/drive/MyDrive/datasets/rgb2ir/trainA', '/content/drive/MyDrive/datasets/rgb2ir/testA', limit=5000)

# IR images: use all
split_and_copy_images(ir_path, '/content/drive/MyDrive/datasets/rgb2ir/trainB', '/content/drive/MyDrive/datasets/rgb2ir/testB')


In [ ]:
pip install wandb torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 13.4 MB/s eta 0:00:00


Dataset class (Tensor dönüştürücü ve unpaired loader)

In [ ]:
from torch.utils.data import Dataset
import os
from PIL import Image

class UnpairedImageDataset(Dataset):
    def __init__(self, rootA, rootB, transform=None):
        self.files_A = os.listdir(rootA)
        self.files_B = os.listdir(rootB)
        self.rootA = rootA
        self.rootB = rootB
        self.transform = transform

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

    def __getitem__(self, index):
        img_A = Image.open(os.path.join(self.rootA, self.files_A[index % len(self.files_A)])).convert("RGB")
        img_B = Image.open(os.path.join(self.rootB, self.files_B[index % len(self.files_B)])).convert("RGB")

        if self.transform:
            img_A = self.transform(img_A)
            img_B = self.transform(img_B)

        return {'A': img_A, 'B': img_B}


In [ ]:
from torch.utils.data import Dataset
import os
from PIL import Image

class UnpairedImageDataset(Dataset):
    def __init__(self, rootA, rootB, transform=None):
        self.files_A = os.listdir(rootA)
        self.files_B = os.listdir(rootB)
        self.rootA = rootA
        self.rootB = rootB
        self.transform = transform

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

    def __getitem__(self, index):
        img_A = Image.open(os.path.join(self.rootA, self.files_A[index % len(self.files_A)])).convert("RGB")
        img_B = Image.open(os.path.join(self.rootB, self.files_B[index % len(self.files_B)])).convert("RGB")

        if self.transform:
            img_A = self.transform(img_A)
            img_B = self.transform(img_B)

        return {'A': img_A, 'B': img_B}


In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = UnpairedImageDataset('/content/drive/MyDrive/datasets/rgb2ir/trainA', '/content/drive/MyDrive/datasets/rgb2ir/trainB', transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
# Add this at the beginning of your notebook or in a cell before the training loop
!pip install pytorch-msssim

import torch
import torch.nn as nn
from tqdm import tqdm
# Ensure Discriminator is defined or imported here if not already
# from your_module import Discriminator # Example import

# Import SSIM loss
from pytorch_msssim import ssim, ms_ssim

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Residual Block (Assuming this is defined correctly elsewhere)
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels)
        )

    def forward(self, x):
        return x + self.block(x)

# Generator (Assuming this is defined correctly elsewhere)
class Generator(nn.Module):
    def __init__(self, num_residual_blocks=9):
        super(Generator, self).__init__()

        model = []

        # Initial conv
        model += [
            nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=3),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        ]

        # Downsampling
        model += [
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True)
        ]

        # Residual Blocks
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(256)]

        # Upsampling
        model += [
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        ]

        # Output layer
        model += [
            nn.Conv2d(64, 3, kernel_size=7, stride=1, padding=3),
            nn.Tanh()
        ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

# Discriminator Class (Assuming this is the correct definition for your model)
class Discriminator(nn.Module):
    def __init__(self, in_channels):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, stride, normalize=True):
            layers = [nn.Conv2d(in_filters, out_filters, kernel_size=3, stride=stride, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(in_channels, 64, stride=2, normalize=False),
            *discriminator_block(64, 128, stride=2),
            *discriminator_block(128, 256, stride=2),
            *discriminator_block(256, 512, stride=2),
            nn.Conv2d(512, 1, kernel_size=3, stride=1, padding=1)
        )

    def forward(self, img):
        # Flatten output for binary classification (real/fake)
        return self.model(img).view(img.size(0), -1)


# Adversarial loss
criterion_GAN = nn.MSELoss().to(device)
# Cycle consistency loss
criterion_cycle = nn.L1Loss().to(device)
# Identity loss
criterion_identity = nn.L1Loss().to(device)

# Define the SSIM loss function (as a module or a functional call)

def ssim_loss(img1, img2):
    # Ensure images are on the same device
    img1 = img1.to(device)
    img2 = img2.to(device)
    # Calculate SSIM. `data_range` depends on your normalization.
    # If images are normalized to [-1, 1], data_range should be 2.0.
    # If images are normalized to [0, 1], data_range should be 1.0.
    # Your transform normalizes to [-1, 1], so data_range=2.0
    # The function returns the SSIM value. For a loss, use 1 - SSIM.
    return 1 - ssim( img1, img2, data_range=2.0, size_average=True) # size_average=True averages SSIM over batch

# Modelleri cihaza taşı
G_AB = Generator(num_residual_blocks=9).to(device)
G_BA = Generator(num_residual_blocks=9).to(device)

# Discriminator modellerini başlatın
D_A = Discriminator(3).to(device) # Assuming RGB discriminator input is 3 channels
D_B = Discriminator(3).to(device) # Assuming IR discriminator input is 3 channels, or adjust if IR is single channel

# Optimizerlar
lr = 0.0002
beta1 = 0.5
optimizer_G = torch.optim.Adam(list(G_AB.parameters()) + list(G_BA.parameters()), lr=lr, betas=(beta1, 0.999))
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(beta1, 0.999))

print("Models and Optimizers initialized.")

# Now, the ssim_loss function is defined and can be used in the training loop.

# with the defined ssim_loss function available.

Using device: cuda
Models and Optimizers initialized.


In [ ]:
# Adversarial loss
criterion_GAN = nn.MSELoss().to(device)
# Cycle consistency loss
criterion_cycle = nn.L1Loss().to(device)
# Identity loss
criterion_identity = nn.L1Loss().to(device)

# Modelleri cihaza taşı
G_AB = Generator(num_residual_blocks=9).to(device)
G_BA = Generator(num_residual_blocks=9).to(device)


D_A = Discriminator(3).to(device)
D_B = Discriminator(3).to(device)

# Optimizerlar
lr = 0.0002
beta1 = 0.5
optimizer_G = torch.optim.Adam(list(G_AB.parameters()) + list(G_BA.parameters()), lr=lr, betas=(beta1, 0.999))
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
print(G_AB)

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = UnpairedImageDataset('/content/drive/MyDrive/datasets/rgb2ir/trainA', '/content/drive/MyDrive/datasets/rgb2ir/trainB', transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
from google.colab import drive
import os
import torch

drive.mount('/content/drive')

checkpoint_dir = '/content/drive/MyDrive/added_histogram_loss'
os.makedirs(checkpoint_dir, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    for i, batch in enumerate(tqdm(dataloader)):
        real_A = batch['A'].to(device)
        real_B = batch['B'].to(device)

        # --------------
        #  Train Generators
        # --------------
        optimizer_G.zero_grad()

        # Identity loss
        loss_id_A = criterion_identity(G_BA(real_A), real_A)
        loss_id_B = criterion_identity(G_AB(real_B), real_B)
        loss_identity = (loss_id_A + loss_id_B) * 5.0

        # GAN loss
        fake_B = G_AB(real_A)
        pred_fake_B = D_B(fake_B)
        valid = torch.ones_like(pred_fake_B, device=device)
        loss_GAN_AB = criterion_GAN(pred_fake_B, valid)

        fake_A = G_BA(real_B)
        pred_fake_A = D_A(fake_A)
        valid = torch.ones_like(pred_fake_A, device=device)
        loss_GAN_BA = criterion_GAN(pred_fake_A, valid)

        # Cycle loss
        recov_A = G_BA(fake_B)
        recov_B = G_AB(fake_A)
        loss_cycle = (criterion_cycle(recov_A, real_A) + criterion_cycle(recov_B, real_B)) * 10.0

        # SSIM loss
        # Pass detached tensors to ssim_loss
        loss_ssim_A = ssim_loss(recov_A.detach(), real_A.detach())
        loss_ssim_B = ssim_loss(recov_B.detach(), real_B.detach())
        loss_ssim = (loss_ssim_A + loss_ssim_B) * 5.0


        # Total generator loss
        loss_G = loss_identity + loss_GAN_AB + loss_GAN_BA + loss_cycle + loss_ssim

        loss_G.backward()
        optimizer_G.step()

        # --------------
        #  Train Discriminators
        # --------------
        optimizer_D_A.zero_grad()
        pred_real_A = D_A(real_A)
        valid = torch.ones_like(pred_real_A, device=device)
        loss_real_A = criterion_GAN(pred_real_A, valid)

        pred_fake_A = D_A(fake_A.detach())
        fake = torch.zeros_like(pred_fake_A, device=device)
        loss_fake_A = criterion_GAN(pred_fake_A, fake)

        loss_D_A = (loss_real_A + loss_fake_A) * 0.5
        loss_D_A.backward()
        optimizer_D_A.step()

        optimizer_D_B.zero_grad()
        pred_real_B = D_B(real_B)
        valid = torch.ones_like(pred_real_B, device=device)
        loss_real_B = criterion_GAN(pred_real_B, valid)

        pred_fake_B = D_B(fake_B.detach())
        fake = torch.zeros_like(pred_fake_B, device=device)
        loss_fake_B = criterion_GAN(pred_fake_B, fake)

        loss_D_B = (loss_real_B + loss_fake_B) * 0.5
        loss_D_B.backward()
        optimizer_D_B.step()

        if i % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}] Batch [{i}] Loss_G: {loss_G.item():.4f}, Loss_D_A: {loss_D_A.item():.4f}, Loss_D_B: {loss_D_B.item():.4f}")

    # Epoch sonu model kaydet
    torch.save(G_AB.state_dict(), f'G_AB_epoch{epoch}.pth')
    torch.save(G_BA.state_dict(), f'G_BA_epoch{epoch}.pth')

  0%|          | 2/4000 [00:00<18:42,  3.56it/s]

Epoch [0/100] Batch [0] Loss_G: 31.2627, Loss_D_A: 0.4242, Loss_D_B: 0.3380


  3%|▎         | 102/4000 [00:14<08:37,  7.54it/s]

Epoch [0/100] Batch [100] Loss_G: 14.2946, Loss_D_A: 0.1874, Loss_D_B: 0.2264


  5%|▌         | 202/4000 [00:29<08:31,  7.43it/s]

...

 95%|█████████▌| 3802/4000 [09:00<00:26,  7.44it/s]

Epoch [17/100] Batch [3800] Loss_G: 4.9314, Loss_D_A: 0.0497, Loss_D_B: 0.0359


 98%|█████████▊| 3902/4000 [09:15<00:13,  7.12it/s]

Epoch [17/100] Batch [3900] Loss_G: 5.2479, Loss_D_A: 0.2076, Loss_D_B: 0.0073


100%|█████████▉| 3984/4000 [09:27<00:02,  7.26it/s]